0. Install and Import Dependencies

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.holistic

<img src="https://www.marktechpost.com/wp-content/uploads/2020/08/Screenshot-2020-08-25-at-10.01.08-PM.png" style="height:500px" >

In [2]:
def calculate_angle(a, b, c):
    """
    a, b, c are arrays with [x, y, z]
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radian = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radian * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle 

In [3]:
cap = cv2.VideoCapture(0)

# Curl counter 
counter = 0
stage = None

# Setup mediapipe instance
with mp_pose.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image, cv2 returns BGR instead of RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # image.flags.writable = False
        
        # Conduct detection using mediapipe
        results = pose.process(image)
        
        # Recolor back to BGR
        # image.flags.writable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Render
            cv2.putText(image, str(round(angle, 3)), 
                        tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                       )
            
            # Curl counter logic
            if angle > 140:
                stage = "down"
            if angle < 50 and stage == 'down':
                stage = "up"
                counter += 1
            
        except Exception as e:
            pass
        
        # Render curl counter
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(img=image, text='REPS', org=(50, 60), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 0), thickness=1, lineType=cv2.LINE_AA)
        cv2.putText(img=image, text=str(counter), org=(10, 60), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 0), thickness=1, lineType=cv2.LINE_AA)
        
        # Render pose detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),  # color of dots
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)  # color of lines
                                 )
        # Render face detection
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_pose.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),  # color of dots
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)  # color of lines
                                 )
        
        # Hand detection
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_pose.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),  # color of dots
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)  # color of lines
                                 )
        
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_pose.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),  # color of dots
                                 mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)  # color of lines
                                 )
        
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 